In [1]:
import torch

import torch.nn as nn
from keras.regularizers import l1,l2
from keras.models import Sequential,Model
from keras.layers import Embedding,Input,Dense,Flatten,Lambda, Multiply, Concatenate


2022-12-15 16:52:07.158290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 16:52:07.258079: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
torch.load('/home/vuhoang181/Documents/code_base/APR-PyTorch/output/bpr.pt')

OrderedDict([('W',
              tensor([[ 0.1027, -0.2571,  0.2280,  ..., -0.3319,  0.1107, -0.0547],
                      [-0.2378,  0.1023, -0.2476,  ..., -0.1183,  0.2373,  0.3497],
                      [ 0.1866, -0.2054, -0.2772,  ..., -0.2186,  0.2774,  0.2513],
                      ...,
                      [ 0.1794, -0.2494,  0.2269,  ..., -0.2170,  0.0641,  0.0161],
                      [ 0.2467, -0.1777,  0.1404,  ..., -0.2994, -0.1175, -0.0339],
                      [-0.1238,  0.1703, -0.1914,  ...,  0.0497,  0.2560, -0.1212]])),
             ('H',
              tensor([[-0.1210, -0.0994, -0.2235,  ..., -0.2647, -0.2383, -0.1061],
                      [-0.1862,  0.1676, -0.3127,  ..., -0.2307, -0.2465, -0.3151],
                      [-0.2041,  0.2428, -0.3161,  ..., -0.2608, -0.1408, -0.2574],
                      ...,
                      [-0.2339, -0.0100,  0.1009,  ..., -0.0604, -0.1932, -0.1700],
                      [ 0.0182,  0.1940, -0.2179,  ..., -0.1539, 

In [ ]:
def get_model(num_users,num_items,mf_dim=10,layers=[10],reg_layers=[0,0,0,0],reg_mf=0):
    num_layer=len(layers)

    user_input=Input(shape=(1,),dtype='int32')
    item_input_pos=Input(shape=(1,),dtype='int32')
    item_input_neg = Input(shape=(1,), dtype='int32')

    ## in this context each point is projected to have 10 dimensional coordinates?
    MF_embedding_user=Embedding(input_dim=num_users,output_dim=mf_dim,embeddings_initializer='random_normal',
                                name='mf_user_embedding',embeddings_regularizer=l2(reg_mf),input_length=1)
    MF_embedding_item = Embedding(input_dim=num_items, output_dim=mf_dim, embeddings_initializer='random_normal',
                                  name='mf_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)
    MLP_embedding_user=Embedding(input_dim=num_users,output_dim=layers[0],embeddings_initializer='random_normal',
                                 name='mlp_user_embedding', embeddings_regularizer=l2(reg_mf),input_length=1)
    MLP_embedding_item = Embedding(input_dim=num_items, output_dim=layers[0], embeddings_initializer='random_normal',
                                   name='mlp_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)

    mf_user_latent=Flatten()(MF_embedding_user(user_input))
    mf_item_latent_pos=Flatten()(MF_embedding_item(item_input_pos))
    mf_item_latent_neg = Flatten()(MF_embedding_item(item_input_neg))

    ## merge = deprecated use keras.layers.Concatenate(axis=-1) instead
    prefer_pos = merge([mf_user_latent, mf_item_latent_pos], mode='mul')
    prefer_neg = merge([mf_user_latent, mf_item_latent_neg], mode='mul')
    ## convert negative layer to negative, lambda layers should be re-written as subclass layer if too complex (eg: multiply by scale?)
    prefer_neg = Lambda(lambda x: -x)(prefer_neg)
    ## basically just merge 2 layer
    mf_vector = merge([prefer_pos, prefer_neg], mode='concat')

    ## flat matrix to an array
    mlp_user_latent=Flatten()(MLP_embedding_user(user_input))
    mlp_item_latent_pos=Flatten()(MLP_embedding_item(item_input_pos))
    mlp_item_latent_neg=Flatten()(MLP_embedding_item(item_input_neg))
    mlp_item_latent_neg=Lambda(lambda x:-x)(mlp_item_latent_neg)
    mlp_vector=merge([mlp_user_latent,mlp_item_latent_pos,mlp_item_latent_neg],mode='concat')
    for idx in range(1,num_layer):
        #set up hidden layer of the network, why tanh and have to regularize?? L1 consider weight, l2 consider square of weight
        layer=Dense(layers[idx],kernel_regularizer=l2(0.0000),activation='tanh',name="layer%d" %idx)
        mlp_vector=layer(mlp_vector)

    ## concatenation of NBPR layer and DNCR layer
    predict_vector=merge([mf_vector,mlp_vector],mode='concat')


    #set up prediction --> final layer, sigmoid activate to give binary output
    prediction=Dense(1,activation='sigmoid',kernel_initializer='lecun_uniform',name='prediction')(predict_vector)
    model=Model(inputs=[user_input,item_input_pos,item_input_neg],outputs=prediction)

    return model

In [5]:
user_input=Input(shape=(1,),dtype='int32')
item_input_pos=Input(shape=(1,),dtype='int32')
item_input_neg = Input(shape=(1,), dtype='int32')

In [7]:
mf_dim=10
layers=[10]
num_layer=len(layers)
reg_layers=[0,0,0,0]
reg_mf=0

In [14]:
MF_embedding_user=Embedding(input_dim=2000,output_dim=mf_dim,embeddings_initializer='random_normal',
                                name='mf_user_embedding',embeddings_regularizer=l2(reg_mf),input_length=1)
MF_embedding_item = Embedding(input_dim=50000, output_dim=mf_dim, embeddings_initializer='random_normal',
                                  name='mf_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)
MLP_embedding_user=Embedding(input_dim=2000,output_dim=layers[0],embeddings_initializer='random_normal',
                                 name='mlp_user_embedding', embeddings_regularizer=l2(reg_mf),input_length=1)
MLP_embedding_item = Embedding(input_dim=50000, output_dim=layers[0], embeddings_initializer='random_normal',
                                   name='mlp_item_embedding',embeddings_regularizer=l2(reg_mf), input_length=1)

In [15]:
mf_user_latent=Flatten()(MF_embedding_user(user_input))
mf_item_latent_pos=Flatten()(MF_embedding_item(item_input_pos))
mf_item_latent_neg = Flatten()(MF_embedding_item(item_input_neg))

2022-12-14 23:27:03.991416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
## merge = deprecated use keras.layers.Concatenate(axis=-1) instead
prefer_pos = Multiply()([mf_user_latent, mf_item_latent_pos])
prefer_neg = Multiply()([mf_user_latent, mf_item_latent_neg])
## convert negative layer to negative, lambda layers should be re-written as subclass layer if too complex (eg: multiply by scale?)
prefer_neg = Lambda(lambda x: -x)(prefer_neg)
## basically just merge 2 layer
mf_vector = Concatenate()([prefer_pos, prefer_neg])

In [27]:
mlp_user_latent=Flatten()(MLP_embedding_user(user_input))
mlp_item_latent_pos=Flatten()(MLP_embedding_item(item_input_pos))
mlp_item_latent_neg=Flatten()(MLP_embedding_item(item_input_neg))
mlp_item_latent_neg=Lambda(lambda x:-x)(mlp_item_latent_neg)
mlp_vector=Concatenate()([mlp_user_latent,mlp_item_latent_pos,mlp_item_latent_neg])
for idx in range(1,num_layer):
    #set up hidden layer of the network, why tanh and have to regularize?? L1 consider weight, l2 consider square of weight
    layer=Dense(layers[idx],kernel_regularizer=l2(0.0000),activation='tanh',name="layer%d" %idx)
    mlp_vector=layer(mlp_vector)

In [37]:
predict_vector=Concatenate()([mf_vector,mlp_vector])


#set up prediction --> final layer, sigmoid activate to give binary output
prediction=Dense(3,kernel_initializer='lecun_uniform',name='prediction')(predict_vector)
model=Model(inputs=[user_input,item_input_pos,item_input_neg],outputs=prediction)

In [38]:
prediction

<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'prediction')>

In [40]:
layer=Dense(1,kernel_regularizer=l2(0.0000),activation='tanh')
mlp_vector=layer(mf_vector)

In [41]:
mlp_vector

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense')>

In [36]:
layers

[10]

In [25]:
num_layer=len(layers)

In [26]:
num_layer

1

In [5]:
W = nn.Parameter(torch.empty(2000, 64))  # User embedding
H = nn.Parameter(torch.empty(10000, 64))  # Item embedding
nn.init.xavier_normal_(W.data)
nn.init.xavier_normal_(H.data)

tensor([[ 0.0071, -0.0003, -0.0036,  ..., -0.0109, -0.0021, -0.0093],
        [-0.0168,  0.0075,  0.0166,  ...,  0.0210,  0.0088,  0.0050],
        [ 0.0069,  0.0208, -0.0190,  ...,  0.0083,  0.0040, -0.0226],
        ...,
        [ 0.0188, -0.0042,  0.0058,  ..., -0.0114,  0.0023,  0.0108],
        [ 0.0068, -0.0273,  0.0137,  ..., -0.0128,  0.0244,  0.0166],
        [ 0.0157, -0.0429, -0.0156,  ..., -0.0312, -0.0007,  0.0045]])

In [34]:
u=W[1,:]
i=H[1,:]
j=H[2,:]

In [26]:
u.retain_grad()
i.retain_grad()
j.retain_grad()

In [22]:
torch.mul(u, i).sum()

tensor(-0.0072, grad_fn=<SumBackward0>)

In [19]:
u

tensor([ 0.0308,  0.0214, -0.0337, -0.0007,  0.0668,  0.0039,  0.0473, -0.0047,
        -0.0129, -0.0313, -0.0083,  0.0202,  0.0343, -0.0094, -0.0246,  0.0225,
        -0.0211,  0.0477, -0.0036, -0.0222, -0.0022, -0.0409, -0.0026, -0.0114,
         0.0423, -0.0686,  0.0195,  0.0083, -0.0347,  0.0004, -0.0054,  0.0049,
        -0.0268, -0.0576, -0.0233, -0.0235, -0.0006,  0.0016, -0.0176,  0.0523,
        -0.0086,  0.0087,  0.0148,  0.0087,  0.0461, -0.0197,  0.0121, -0.0110,
        -0.0299, -0.0086, -0.0063, -0.0063,  0.0055, -0.0220, -0.0304, -0.0149,
        -0.0629,  0.0318,  0.0386, -0.0172, -0.0283, -0.0047, -0.0392, -0.0386],
       grad_fn=<SliceBackward0>)

In [23]:
import torch.nn.functional as F


In [24]:
F.logsigmoid(torch.mul(u, i).sum()).sum()

tensor(-0.6968, grad_fn=<SumBackward0>)

In [29]:
u

tensor([ 0.0308,  0.0214, -0.0337, -0.0007,  0.0668,  0.0039,  0.0473, -0.0047,
        -0.0129, -0.0313, -0.0083,  0.0202,  0.0343, -0.0094, -0.0246,  0.0225,
        -0.0211,  0.0477, -0.0036, -0.0222, -0.0022, -0.0409, -0.0026, -0.0114,
         0.0423, -0.0686,  0.0195,  0.0083, -0.0347,  0.0004, -0.0054,  0.0049,
        -0.0268, -0.0576, -0.0233, -0.0235, -0.0006,  0.0016, -0.0176,  0.0523,
        -0.0086,  0.0087,  0.0148,  0.0087,  0.0461, -0.0197,  0.0121, -0.0110,
        -0.0299, -0.0086, -0.0063, -0.0063,  0.0055, -0.0220, -0.0304, -0.0149,
        -0.0629,  0.0318,  0.0386, -0.0172, -0.0283, -0.0047, -0.0392, -0.0386],
       grad_fn=<SliceBackward0>)

In [62]:
mlp_vector=torch.stack((u,i,j))

In [47]:
Dense(1,kernel_regularizer=l2(0.0000),activation='tanh',name='final')(mlp_vector)

AttributeError: 'torch.Size' object has no attribute 'rank'

In [77]:
m = torch.nn.Linear(mlp_vector.shape[1], 1)

In [51]:
import torch
import torch.nn as nn

x = torch.tensor([[1.0, -1.0],
                  [0.0,  1.0],
                  [0.0,  0.0]])

in_features = x.shape[1]  # = 2
out_features = 2

m = nn.Linear(in_features, out_features)

In [87]:
m(mlp_vector)

tensor([[-0.0586],
        [-0.0656],
        [-0.0755]], grad_fn=<AddmmBackward0>)

In [90]:
m1 = nn.Linear(64, 32)
m2 = nn.Linear(32,1)

In [91]:
m2(m1(mlp_vector))

tensor([[-0.0560],
        [-0.0568],
        [-0.0535]], grad_fn=<AddmmBackward0>)

In [92]:
linear_relu_stack = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

In [95]:
linear_relu_stack(mlp_vector)

tensor([[0.1656],
        [0.1633],
        [0.1632]], grad_fn=<AddmmBackward0>)

In [96]:
linear_relu_stack(mlp_vector)[1]

tensor([0.1633], grad_fn=<SelectBackward0>)